<a href="https://colab.research.google.com/github/Reynxzz/capstone-ipb/blob/main/Capstone_IPB_Making_Dataset_%26_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Dataset

STEP bikin dataset:
1. Bikin random point di google earth sekitaran IPB
2. Export latitude dan longitude dari point2 itu
3. Generate random route (cyclic) dengan start Agrimart dan Rektorat
4. Sesuaikan distancenya (max 3 km dan max 5 km)
5. Ambil data elevation per point subset tersebut pake Open Topodata API
6. Sesuaikan kesulitannya pake data elevasi (Easy, Medium, Hard) --> lagi disini
7. Sesuaikan dengan pemandangan

Expected dataset:

| Route | (lat,lon) | Distance | Duration | Elevation Max | Elevation Min | Elevation Difference | Difficulty | Scenery | Preview(image) |

###Import CSV datasets from Google Earth

In [85]:
import csv

# Open the CSV file for reading
with open('/content/drive/My Drive/Capstone IPB/Sample_Point_IPB_Datasets.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    # Create a list of tuples containing latitude and longitude
    coordinates = [(float(row['Y']), float(row['X'])) for row in reader]

In [89]:
coordinates

[(-6.56084284005594, 106.730112484034),
 (-6.55999627461358, 106.730309831525),
 (-6.55984667674852, 106.731220037341),
 (-6.55874510940651, 106.731065282078),
 (-6.55808372282102, 106.731483631852),
 (-6.55786891037331, 106.728888601569),
 (-6.55808537536097, 106.727585030038),
 (-6.55971469863472, 106.728055106482),
 (-6.56086040150612, 106.727948585279),
 (-6.56083181843221, 106.726300134217),
 (-6.56102125994555, 106.725720407015),
 (-6.56067833579035, 106.725457774325),
 (-6.56015620105988, 106.724135061428),
 (-6.56015917306977, 106.722770681276),
 (-6.55868734718743, 106.727082130237),
 (-6.55704480044615, 106.726596908089),
 (-6.55624285746221, 106.726577711467),
 (-6.55606449789467, 106.72504994857),
 (-6.55702792189835, 106.724931035447),
 (-6.5561681108102, 106.724086099391),
 (-6.55484928973291, 106.724258038487),
 (-6.5548331700862, 106.72306209134),
 (-6.55390098457591, 106.724776383285),
 (-6.55400818626206, 106.727729858342),
 (-6.55573692463267, 106.728152616859),
 (-6

Create dataframe

In [5]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Capstone IPB/Sample_Point_IPB_Datasets.csv')

In [6]:
# remove uneccessary column
df = df.drop(df.columns[-1], axis=1)

# change column name
df = df[['Name', 'Y', 'X']]
df = df.rename(columns={'Name': 'name', 'X': 'longitude', 'Y': 'latitude'})
df

,name,latitude,longitude
0,ATM,-6.561020,106.731518
1,GWW,-6.560843,106.730112
2,Pertigaan GWW,-6.559996,106.730310
3,Starting Point 1 - Agrimart,-6.559847,106.731220
4,Kopcen,-6.558745,106.731065
...,...,...,...
59,Belakang Faperta,-6.556731,106.722454
60,RS Hewan,-6.557114,106.720587
61,Pintu Masuk ke Kiri,-6.557232,106.719038
62,Agatis 1,-6.558186,106.719918


Visualize Dataframe

In [7]:
import folium
# create a map centered on the first location
m_raw = folium.Map(location=[df['latitude'][0], df['longitude'][0]], zoom_start=10)

# add markers for each location in the DataFrame
for index, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(m_raw)

# display the map
m_raw

###Generate Routes

In [11]:
def get_elevations(coordinates):
    """Lihat dokumentasi disini bro https://www.opentopodata.org/"""

    # Build the API URL for the OpenTopo API
    url = "https://api.opentopodata.org/v1/aster30m"

    # Make the API call for each coordinate and extract the elevation data
    elevations = []
    for coord in coordinates:
        lat, lon = coord
        query = f"{url}?locations={lat},{lon}"
        response = requests.get(query)
        if response.status_code == 200:
            data = response.json()
            elevation = data["results"][0]["elevation"]
            elevations.append(elevation)
        else:
            elevations.append(None)

    # Return the list of elevations
    return elevations

Bikin rute random

In [114]:
import random
import json
import requests
import time

def routes_maker(total_point, max_distance, threshold=500):
    subset = []
    # define start and end coordinates
    start = (-6.5610202378231, 106.731518082304)
    end = (-6.5610202378231, 106.731518082304)

    if start in coordinates:
      coordinates.remove(start)

    # randomly select a subset of coordinates
    subset = random.sample(coordinates, k=total_point) # k is the number of coordinates in the subset

    # add the start and end coordinates to the beginning and end of the subset
    subset.insert(0, start)
    subset.append(end)

    api_url = 'https://api.openrouteservice.org/v2/directions/foot-walking'
    api_key = '5b3ce3597851110001cf6248d4725ac662f8461eb589679758fb20ea'

    locations = [list(r) for r in subset]
    total_distance = 0
    total_duration = 0

    for i in range(len(locations)-1):
        # Define the start and end coordinates for the API call
        start_coords = f"{locations[i][1]},{locations[i][0]}"
        end_coords = f"{locations[i+1][1]},{locations[i+1][0]}"
        
        # Define the API call parameters, including the start and end coordinates, the API key, and the travel mode (walking)
        params = {
            'api_key': api_key,
            'start': start_coords,
            'end': end_coords,
            'profile': 'foot-walking',
        }
        
        # Make the API call and parse the response as JSON
        response = requests.get(api_url, params=params)
        try:
          data = json.loads(response.text)
          # print(data)
          # Extract the walking distance and duration from the response
          distance = data['features'][0]['properties']['segments'][0]['distance']
          duration = data['features'][0]['properties']['segments'][0]['duration']
          # elevation = data['features'][0]['properties']['segments'][0]['steps'][0]['elevation']
        except:
          print("Rate limit exceeded. Retrying in 30 seconds...")
          time.sleep(30)
          return routes_maker(total_point, max_distance)

        # print(f"Distance between locations {i+1} and {i+2}: {distance:.2f} meters")

        # total distance and duration
        total_distance += distance
        total_duration += duration
        
        # visualize the route
        m = folium.Map(location=start, zoom_start=14)
        folium.PolyLine(locations=[list(reversed(coord)) 
                              for coord in 
                              data['features'][0]['geometry']['coordinates']]).add_to(m)

    # print total distance and duration
    print(f"Total distance: {total_distance:.2f} meters")
    print(f"Total duration: {total_duration:.2f} seconds")
    
    # add marker to map
    folium.Marker(location=start, popup='Start here', icon=folium.Icon(color='green', prefix='fa', icon='running')).add_to(m)
    folium.Marker(location=locations[len(locations) // 2], popup='Finish here (turn around)', icon=folium.Icon(color='blue',prefix='fa', icon='undo')).add_to(m)

    # get elevation of the subset
    subset_elevations = get_elevations(subset)

    # get max, min, difference elevation of the subset
    max_elevation = max(subset_elevations)
    min_elevation = min(subset_elevations)
    elevation_difference = max_elevation - min_elevation

    # if requirements not satisfied, run the function again!
    if abs(total_distance - max_distance) <= threshold:
      return total_distance, total_duration, subset, subset_elevations, max_elevation, min_elevation, elevation_difference, m  
    else:      
      return routes_maker(total_point, max_distance)

In [137]:
#test the function
total_distance, total_duration, subset, subset_elevations, max_elevation, min_elevation, elevation_difference, m = routes_maker(2, 3000)

Total distance: 2914.30 meters
Total duration: 2098.30 seconds


In [140]:
# Initialize an empty list to store the output
output_list = []

# Call the function 10 times and append the output to the list
for i in range(10):
    print(f'=== Iteration {i+1} ===')
    total_distance, total_duration, subset, subset_elevations, max_elevation, min_elevation, elevation_difference, m = routes_maker(2, 5000)
    output_list.append((total_distance, total_duration, subset, subset_elevations, max_elevation, min_elevation, elevation_difference, m))
    

# Print the list of output
print(output_list)

=== Iteration 1 ===
Total distance: 7208.00 meters
Total duration: 5189.60 seconds
Total distance: 3035.70 meters
Total duration: 2185.70 seconds
Total distance: 4206.60 meters
Total duration: 3028.70 seconds
Total distance: 4229.80 meters
Total duration: 3045.40 seconds
Total distance: 4120.50 meters
Total duration: 2966.60 seconds
Total distance: 4053.40 meters
Total duration: 2918.50 seconds
Total distance: 4460.50 meters
Total duration: 3211.50 seconds
Total distance: 1762.20 meters
Total duration: 1268.70 seconds
Total distance: 6769.20 meters
Total duration: 4873.90 seconds
Total distance: 1330.20 meters
Total duration: 957.80 seconds
Total distance: 2740.60 meters
Total duration: 1973.30 seconds
Total distance: 4539.60 meters
Total duration: 3268.40 seconds
=== Iteration 2 ===
Total distance: 5316.00 meters
Total duration: 3827.50 seconds
=== Iteration 3 ===
Rate limit exceeded. Retrying in 30 seconds...
Total distance: 5876.40 meters
Total duration: 4231.00 seconds
Total distan

In [141]:
len(output_list)

10

In [142]:
output_list[0]

(4539.6,
 3268.4,
 [(-6.5610202378231, 106.731518082304),
  (-6.55484928973291, 106.724258038487),
  (-6.54996484285961, 106.723928058488),
  (-6.5610202378231, 106.731518082304)],
 [182.0, 183.0, 142.0, 182.0],
 183.0,
 142.0,
 41.0,
 <folium.folium.Map at 0x7f8cbdff32e0>)

In [ ]:
# total_distances, m, subset, total_duration = routes_maker(2, 5000)

In [ ]:
# routes = [total_distances] + [total_duration] + subset 
# routes

In [ ]:
# subset_elevations = get_elevations(output_list[0][2])
# subset_elevations

In [ ]:
# max_elevation = max(subset_elevations)
# min_elevation = min(subset_elevations)
# elevation_difference = max_elevation - min_elevation

# max_elevation, min_elevation, elevation_difference

Initial Dataframe

In [100]:
import random
import string
import numpy as np

In [101]:
# Generate routes ID
id_length = 8
characters = string.ascii_letters + string.digits

In [143]:
rows = []
for i in range(10):
    row_data = output_list[i % len(output_list)]

    row_dict = {'id': ''.join(random.choice(characters) for j in range(id_length)),
                'route_coordinate (lat,lon)': row_data[2],
                'distance_type': 5 if row_data[0] < 500 + row_data[0] or row_data[0] > row_data[0] - 500 else 3,
                'distance (m)': row_data[0],
                'duration (s)': row_data[1],
                'max_elevation (m)': row_data[4],
                'min_elevation (m)': row_data[5],
                'elevation_diff (m)': row_data[6],
                'difficulty': None,
                'scenery': None,
                'map_preview': row_data[7]}
    rows.append(row_dict)

df_routes = pd.DataFrame(rows)

In [144]:
df_routes

,id,"route_coordinate (lat,lon)",distance_type,distance (m),duration (s),max_elevation (m),min_elevation (m),elevation_diff (m),difficulty,scenery,map_preview
0,8JHBdOMX,"[(-6.5610202378231, 106.731518082304), (-6.554...",5,4539.6,3268.4,183.0,142.0,41.0,None,None,<folium.folium.Map object at 0x7f8cbdff32e0>
1,C2c3UPX7,"[(-6.5610202378231, 106.731518082304), (-6.559...",5,5316.0,3827.5,184.0,141.0,43.0,None,None,<folium.folium.Map object at 0x7f8cbe29c640>
2,V3s1cp8S,"[(-6.5610202378231, 106.731518082304), (-6.552...",5,5144.4,3703.9,183.0,152.0,31.0,None,None,<folium.folium.Map object at 0x7f8cbe1b90c0>
3,GL5s928R,"[(-6.5610202378231, 106.731518082304), (-6.551...",5,4972.9,3580.4,183.0,152.0,31.0,None,None,<folium.folium.Map object at 0x7f8cbe109630>
4,EAQ5o90s,"[(-6.5610202378231, 106.731518082304), (-6.552...",5,4637.4,3338.9,183.0,167.0,16.0,None,None,<folium.folium.Map object at 0x7f8cbe1b9db0>
5,ULZMecog,"[(-6.5610202378231, 106.731518082304), (-6.550...",5,4504.9,3243.5,189.0,165.0,24.0,None,None,<folium.folium.Map object at 0x7f8cbe262380>
6,hSFlZkFB,"[(-6.5610202378231, 106.731518082304), (-6.553...",5,4973.0,3580.5,182.0,152.0,30.0,None,None,<folium.folium.Map object at 0x7f8cbe29eb30>
7,3kuoXz6U,"[(-6.5610202378231, 106.731518082304), (-6.549...",5,5407.5,3893.3,182.0,152.0,30.0,None,None,<folium.folium.Map object at 0x7f8cbe10ba60>
8,XAN3LLiJ,"[(-6.5610202378231, 106.731518082304), (-6.560...",5,4809.3,3462.6,189.0,148.0,41.0,None,None,<folium.folium.Map object at 0x7f8cbe1b8b50>
9,8JwXO5Wg,"[(-6.5610202378231, 106.731518082304), (-6.552...",5,4828.3,3476.2,182.0,167.0,15.0,None,None,<folium.folium.Map object at 0x7f8cbe263e80>


Visualisasi rute

In [147]:
df_routes["map_preview"][0]

Experiment

In [ ]:
!pip install osmnx

In [ ]:
import osmnx as ox

# Define the location of interest using an address or place name
location_name = 'San Francisco, California'

# Define the distance (in meters) around the location to retrieve OSM data for
distance = 500

# Download and construct a network from OSM data within the specified distance from the location
G = ox.graph_from_address(location_name, distance, network_type='all')

# Calculate the area of the network in square meters
area = ox.set_graph_crs(G, 'EPSG:4326').unary_union.area

# Calculate the amount of tree cover within the network
trees = ox.geometries_from_address(location_name, tags={'natural': 'tree'})
tree_cover = trees.geometry.unary_union.area / area

print(f"Total area: {area:.2f} square meters")
print(f"Tree cover: {tree_cover:.2%}")

RuntimeError: ignored

AttributeError: ignored